In [17]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key='0b63f17479414114b7006e9d1ed56f87',
    api_version="2024-07-01-preview",
    azure_endpoint='https://soikit-gpt4-mini.openai.azure.com/'
)

# Upload a file with a purpose of "batch"
file = client.files.create(
    file=open("../test.jsonl", "rb"),
    purpose="batch"
)

print(file.model_dump_json(indent=2))
file_id = file.id


{
  "id": "file-7b235269712c45ff9543e999920e46d0",
  "bytes": 886,
  "created_at": 1724666325,
  "filename": "test.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "pending",
  "status_details": null
}


In [12]:
# Wait until the uploaded file is in processed state
import time
import datetime 

status = "pending"
while status != "processed":
    time.sleep(2)
    file_response = client.files.retrieve(file_id)
    status = file_response.status
    print(f"{datetime.datetime.now()} File Id: {file_id}, Status: {status}")

2024-08-26 17:51:37.601734 File Id: file-479b38f603464bc092e7410ca516c2c0, Status: processed


In [13]:
# Submit a batch job with the file
batch_response = client.batches.create(
    input_file_id=file_id,
    endpoint="/chat/completions",
    completion_window="24h",
)

# Save batch ID for later use
batch_id = batch_response.id

print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_db97b881-4c8e-4314-b726-de990241b3d0",
  "completion_window": "24h",
  "created_at": 1724665918,
  "endpoint": "/chat/completions",
  "input_file_id": "file-479b38f603464bc092e7410ca516c2c0",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1724752318,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


In [14]:
import time
import datetime 

status = "validating"
while status not in ("completed", "failed", "canceled"):
    time.sleep(60)
    batch_response = client.batches.retrieve(batch_id)
    print(batch_response)
    status = batch_response.status
    print(f"{datetime.datetime.now()} Batch Id: {batch_id},  Status: {status}")

Batch(id='batch_db97b881-4c8e-4314-b726-de990241b3d0', completion_window='24h', created_at=1724665918, endpoint='/chat/completions', input_file_id='file-479b38f603464bc092e7410ca516c2c0', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1724752318, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
2024-08-26 17:53:44.205195 Batch Id: batch_db97b881-4c8e-4314-b726-de990241b3d0,  Status: validating
Batch(id='batch_db97b881-4c8e-4314-b726-de990241b3d0', completion_window='24h', created_at=1724665918, endpoint='/chat/completions', input_file_id='file-479b38f603464bc092e7410ca516c2c0', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='model_mismatch', line=None, message="The provid

In [15]:
print(batch_response.model_dump_json(indent=2))


{
  "id": "batch_db97b881-4c8e-4314-b726-de990241b3d0",
  "completion_window": "24h",
  "created_at": 1724665918,
  "endpoint": "/chat/completions",
  "input_file_id": "file-479b38f603464bc092e7410ca516c2c0",
  "object": "batch",
  "status": "failed",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": {
    "data": [
      {
        "code": "model_mismatch",
        "line": null,
        "message": "The provided model deployment 'REPLACE-WITH-MODEL-DEPLOYMENT-NAME' does not exist in the AOAI resource.",
        "param": null
      },
      {
        "code": "invalid_request",
        "line": null,
        "message": "The request is invalid.",
        "param": null
      }
    ],
    "object": "list"
  },
  "expired_at": null,
  "expires_at": 1724752318,
  "failed_at": 1724666077,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed

In [9]:
import json

file_response = client.files.content(batch_response.output_file_id)
raw_responses = file_response.text.strip().split('\n')  

for raw_response in raw_responses:  
    json_response = json.loads(raw_response)  
    formatted_json = json.dumps(json_response, indent=2)  
    print(formatted_json)

ValueError: Expected a non-empty value for `file_id` but received None